# Training the model

In [ ]:
#!pip install -U hopsworks --quiet

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

/Users/lassehylleberg/anaconda3/envs/MLOPS/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/550037
2024-05-06 16:54:29,702 WARNING: using legacy validation callback


Connected. Call `.close()` to terminate connection gracefully.


In [3]:
feature_view = fs.get_feature_view("user_beer_rating_feature_view", 1)

## <span style="color:#ff5f27;"> ✨ Load Training Data </span>

In [4]:
import pandas as pd

In [9]:
import pandas as pd
import pickle

# Load the training data
with open('X_train.pkl', 'rb') as file:
    X_train = pickle.load(file)

with open('y_train.pkl', 'rb') as file:
    y_train = pickle.load(file)

## <span style="color:#ff5f27;"> 🏃 Train Model</span>

In [12]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1362589 entries, 0 to 1362588
Columns: 104 entries, beer_name to datetime
dtypes: datetime64[ns](2), float64(5), int64(4), object(4), uint8(89)
memory usage: 271.6+ MB


In [11]:
#!pip install -U xgboost --quiet
import xgboost as xgb

# Convert floating-point values in y_train to integers
y_train = y_train.round().astype(int)

# Train the classifier again
clf.fit(X_train, y_train)

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:beer_name: object, beer_beerid: object, review_time: datetime64[ns], review_profilename: object, review_text: object, datetime: datetime64[ns]